# Consolidación de Bases de Datos

En este punto se tienen tres bases de datos con información de Colegios en Bogotá y sus diferentes teléfonos de contacto. El objetivo de este Notebook es permitir la consolidación de las bases de datos para generar un único resultado de teléfonos nuevos para las instituciones educativas.

## Instalación de Librerías

Para este caso únicamente se requiere instalar `fuzzywuzzy` y `python-Lenshtein`, que se utilizarán para realizar el emparejamiento de instituciones educativas a partir de nombres.

In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [2]:
import ast
import re

import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process

# Conexión a Información

Actualmente tenemos información de tres fuentes:

1. Base de Datos de Colegios (Estudio)
2. Base de Datos de Red Académica (Scraper 1)
3. Base de Datos de Colombia Data (Scraper 2)

El objetivo de este Notebook es combinar por código DANE (inicialmente) y por nombre (de manera auxiliar) los colegios de la base inicial, con las dos bases scrapeadas.

## Base Principal

Contiene la información de los colegios objetivo para la búsqueda. Algunos cuentan con información del DANE en el campo `id_dane`. Sin embargo, el 15.8% no tiene información de código DANE y requerirán ser emparejados por nombre. En total son 3033 colegios para emparejar.

In [3]:
base_principal = pd.read_csv('drive/MyDrive/base_principal.csv')

In [4]:
base_principal.head()

,nombre,direccion,telefono,base,id_dane,id_sirbe
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,KR 51 # 16 - 64 SUR,7209631 -- 2028884,DANE,1.110010e+11,NaN
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,KR 70 B # 24 B - 35 SUR,4140218 -- 4140223,DANE,1.110010e+11,NaN
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,KR 18 A # 187 - 67/65,6714615 -- 3043997208,DANE,1.110010e+11,NaN
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,CL 15 A SUR # 6 - 25 ESTE,6012464434,DANE,1.110010e+11,NaN
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,CL 90 A # 95 D - 56 -- KR 95 G # 90 A - 40,4345708 -- 4339567 -- 4339468,DANE,1.110010e+11,NaN


In [5]:
base_principal['id_dane'] = pd.to_numeric(base_principal['id_dane'])

In [6]:
# Proporción de Colegios sin código DANE
base_principal.id_dane.isna().sum()/len(base_principal)

0.1582591493570722

In [7]:
# No se encuentra información de colegios duplicados.
len(base_principal)-len(base_principal.drop_duplicates())

0

In [8]:
def clean_and_split_phones(phone):
  """
  Extrae la información de números [de teléfono] de una cadena de texto.
  """
  if pd.isna(phone):
      return np.nan

  split_numbers = re.split(r'\s+|/|-|,', phone)


  return [re.sub(r'\D', '', num) for num in split_numbers if num]

In [9]:
# Se crea una lista de teléfonos
base_principal['telefono'] = base_principal['telefono'].apply(clean_and_split_phones)

In [10]:
base_principal.head()

,nombre,direccion,telefono,base,id_dane,id_sirbe
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,KR 51 # 16 - 64 SUR,"[7209631, 2028884]",DANE,1.110010e+11,NaN
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,KR 70 B # 24 B - 35 SUR,"[4140218, 4140223]",DANE,1.110010e+11,NaN
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,KR 18 A # 187 - 67/65,"[6714615, 3043997208]",DANE,1.110010e+11,NaN
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,CL 15 A SUR # 6 - 25 ESTE,[6012464434],DANE,1.110010e+11,NaN
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,CL 90 A # 95 D - 56 -- KR 95 G # 90 A - 40,"[4345708, 4339567, 4339468]",DANE,1.110010e+11,NaN


In [11]:
base_principal.base.value_counts()

DANE     2545
SIRBE     469
Name: base, dtype: int64

## Base Datos Colombia

Contiene la información de todos los colegios listados en la página de [Datos Clombia](https://datoscolombia.com/escuelas-colegios), filtrados para ser únicamente los disponibles en Bogotá. Cuenta con un total de 3421 colegios de la capital (más que en la base principal).

In [12]:
colegios_bogota = pd.read_csv('drive/MyDrive/complete_datos_colombia_information_bogota.csv')

In [13]:
colegios_bogota.head()

,id_institución,nombre_establecimiento,nombre_sede,estado_sede,código_establecimiento,código_sede,zona,telefono,dirección,ciudad,departamento,modelos,grados,url,name,page_num
0,106399.0,Col Cambridge,Col Cambridge,Cierre Temporal,3.110010e+11,3.110010e+11,Urbana,2158649,Tv 14 A 115 06,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/10...,Col Cambridge,1
1,113157.0,Jard Inf Margy,Jard Inf Margy,Cierre Definitivo,3.110011e+11,3.110011e+11,Urbana,3680192 / 3690961,Cr. 45 No.22c-51,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Jard Inf Margy,5
2,113374.0,Inst Scalas,Inst Scalas,Cierre Temporal,3.110011e+11,3.110011e+11,Urbana,2398028,Cr. 6 No. 25-60 Sur,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Inst Scalas,5
3,116203.0,Colegio Tibabuyes Universal (Ied),Sede C,Nuevo-Activo,1.110010e+11,1.110011e+11,Urbana,NaN,Kr 107 139 78,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Sede C,5
4,116593.0,Col Gimn Carolin,Col Gimn Carolin,Cierre Temporal,3.110011e+11,3.110011e+11,Urbana,7689768,Kr 6 C 90 50 Sur,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Col Gimn Carolin,5


La principal información a extraer es el teléfono de contacto, el cual apenas es nulo para el 5% de los colegios.

In [14]:
colegios_bogota.telefono.isna().sum()/len(colegios_bogota)

0.05203156971645718

In [15]:
colegios_bogota.rename({
    'telefono':'nuevo_telefono',
}, axis = 1, inplace = True)

In [16]:
colegios_bogota['código_establecimiento'] = pd.to_numeric(colegios_bogota['código_establecimiento'])

In [17]:
# Se realiza la limpieza del teléfono de Colegios Bogotá
colegios_bogota['nuevo_telefono'] = colegios_bogota['nuevo_telefono'].apply(clean_and_split_phones)

colegios_bogota.head()

,id_institución,nombre_establecimiento,nombre_sede,estado_sede,código_establecimiento,código_sede,zona,nuevo_telefono,dirección,ciudad,departamento,modelos,grados,url,name,page_num
0,106399.0,Col Cambridge,Col Cambridge,Cierre Temporal,3.110010e+11,3.110010e+11,Urbana,[2158649],Tv 14 A 115 06,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/10...,Col Cambridge,1
1,113157.0,Jard Inf Margy,Jard Inf Margy,Cierre Definitivo,3.110011e+11,3.110011e+11,Urbana,"[3680192, 3690961]",Cr. 45 No.22c-51,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Jard Inf Margy,5
2,113374.0,Inst Scalas,Inst Scalas,Cierre Temporal,3.110011e+11,3.110011e+11,Urbana,[2398028],Cr. 6 No. 25-60 Sur,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Inst Scalas,5
3,116203.0,Colegio Tibabuyes Universal (Ied),Sede C,Nuevo-Activo,1.110010e+11,1.110011e+11,Urbana,NaN,Kr 107 139 78,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Sede C,5
4,116593.0,Col Gimn Carolin,Col Gimn Carolin,Cierre Temporal,3.110011e+11,3.110011e+11,Urbana,[7689768],Kr 6 C 90 50 Sur,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Col Gimn Carolin,5


Se encuentra que varios colegios tienen el mismo código DANE para sus diferentes sedes. En esta aproximiación se considera unificar los teléfoos disponibles de todas las sedes.

In [18]:
colegios_bogota_tel_and_id = colegios_bogota.groupby('código_establecimiento')['nuevo_telefono'].agg(
    lambda x: list(set(number for sublist in x if sublist is not np.nan for number in sublist))
).reset_index()

In [19]:
colegios_bogota_tel_and_id = colegios_bogota[['código_establecimiento', 'nuevo_telefono']].copy()

In [20]:
colegios_bogota_tel_and_id

,código_establecimiento,nuevo_telefono
0,3.110010e+11,[2158649]
1,3.110011e+11,"[3680192, 3690961]"
2,3.110011e+11,[2398028]
3,1.110010e+11,NaN
4,3.110011e+11,[7689768]
...,...,...
3416,1.110010e+11,[3636848]
3417,1.110010e+11,"[2068504, 3639119]"
3418,1.110010e+11,"[3678402, 3644235]"
3419,1.110010e+11,[2397232]


In [21]:

def merge_telefonos(series):
    # Filter out NaNs and flatten the list
    filtered_list = [item for sublist in series if sublist == sublist for item in sublist]
    return filtered_list

grouped = colegios_bogota_tel_and_id.groupby('código_establecimiento')['nuevo_telefono'].agg(merge_telefonos)
colegios_bogota_tel_and_id = grouped.to_frame()
colegios_bogota_tel_and_id.reset_index(inplace=True)

In [22]:
colegios_bogota_tel_and_id

,código_establecimiento,nuevo_telefono
0,1.110010e+11,"[5650250, 7209631, 2022340, 2387091]"
1,1.110010e+11,"[4140218, 4140223, 2048314, 2048323]"
2,1.110010e+11,"[6704916, 6745973, 6749258]"
3,1.110010e+11,"[4345708, 4339567, 2295726]"
4,1.110010e+11,[2397516]
...,...,...
3034,4.117690e+11,[6836755]
3035,4.118480e+11,"[6760284, 6765083, 6761480]"
3036,5.111002e+11,[6623999]
3037,5.511001e+11,[2400595]


A continuación se presenta un ejemplo de un colegio multi-sede.

In [23]:
colegios_bogota[colegios_bogota.código_establecimiento == 211001076346.0] # Hay colegios con muchas sedes y el mismo nombre

,id_institución,nombre_establecimiento,nombre_sede,estado_sede,código_establecimiento,código_sede,zona,nuevo_telefono,dirección,ciudad,departamento,modelos,grados,url,name,page_num
66,82583.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Peñaliza,Antiguo-Activo,2.110011e+11,2.110010e+11,Rural,NaN,Corregimiento Betania,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4,5",https://datoscolombia.com/escuelas-colegios/82...,Cent Educ Dist Rural Peñaliza,998
561,86990.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Adelina Gutierrez Palacios,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,[4544416],Km 8 Via Correg Betania,"Bogotá, D.C.","Bogotá, D.C.","Programa Para Jóvenes En Extraedad Y Adultos,E...","-1,0,1,2,3,4,5,6,7,8,9,21,22,23,24,25,26,99",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Adelina Gutierrez Palacios,1218
562,86991.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural El Tabaco,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Betania,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural El Tabaco,1218
563,86992.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Las Animas,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Nazareth,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4,5",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Las Animas,1218
564,86993.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Santa Rosa De Nazareth,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Nazareth,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4,5",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Santa Rosa De Nazareth,1218
565,86994.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Las Sopas,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Nazareth,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4,5",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Las Sopas,1218
566,86995.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Santa Rosa De Bodegas,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Nazareth,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,4",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Santa Rosa De Bodegas,1218
567,86996.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Los Rios,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Nazareth,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4,5",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Los Rios,1218
568,86997.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural Nazareth,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Nazareth,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural Nazareth,1218
569,86998.0,Colegio Camp Jaime Garzon (Ied),Cent Educ Dist Rural El Raizal,Antiguo-Activo,2.110011e+11,2.118500e+11,Rural,NaN,Corregimiento Betania,"Bogotá, D.C.","Bogotá, D.C.",Educación Tradicional,"-2,-1,0,1,2,3,4",https://datoscolombia.com/escuelas-colegios/86...,Cent Educ Dist Rural El Raizal,1218


Adicionalmente, se unifica el nombre de la sede con el nombre del colegio en caso de que estos existan.

In [24]:
#colegios_bogota.loc[colegios_bogota.nombre_establecimiento != colegios_bogota.nombre_sede, 'nombre_establecimiento'] = colegios_bogota.nombre_establecimiento + ' - ' + colegios_bogota.nombre_sede

## Red Académica

Contiene información de 292 colegios de Bogotá, algunos identificados con código DANE. Fue la primera fuente encontrada, aunque es bastante pequeña.

In [25]:
red_academica = pd.read_csv('drive/MyDrive/red_academica_information.csv')

In [26]:
red_academica.head()

,jornada,dirección,correo,teléfono,horario de atención,nombre,idx,dane
0,"Mañana, Tarde, Nocturna",Carrera 18A # 187-75,insdiaquileoparrav1@educacionbogota.edu.co,6745973 - 6745973,8:00 a.m a 4:00 p.m.,Colegio Aquileo Parra,0,NaN
1,Jornada única,km 13 de la vía Bogotá - Choachí,escdielverjonbajoe2@redp.edu.co,NaN,NaN,IED Colegio El Verjon,0,NaN
2,"Mañana, Tarde",Calle 42 A Sur # 12 A - 27 Este,escdialtamirasuror4@educacionbogota.edu.co,2068596 / 3043999116,M y V 9:00 a.m. a 11:00 a.m. - 2:00 p.m. a 4:0...,Colegio Altamira Sur Oriental (IED),0,NaN
3,"Mañana, Tarde",Calle 24A Sur No 1A- 95 Este,cedmontebello4@educacionbogota.edu.co,2067680 - 3002067382,8:00 a.m. - 4:00 p.m.,Colegio Montebello (IED),0,NaN
4,"Mañana, Tarde",Diagonal 34 Sur # 2A-05 Este Barrio: Atenas.,NaN,2069970,Lunes a Viernes de 8:00 am - 4:30pm.,Colegio Atenas (IED),0,NaN


In [27]:
len(red_academica)

292

In [28]:
# Se tiene 19% de los datos sin información de teléfono para contacto
red_academica.teléfono.isna().sum()/len(red_academica)

0.1917808219178082

In [29]:
# Nuevamente es necesario limpiar los teléfonos
red_academica['red_telefono'] = red_academica['teléfono'].apply(clean_and_split_phones)

Los datos de contacto en la Red Académica a menudo presentan inconsistencias, especialmente en la forma de escribir los números telefónicos. Por ejemplo, algunos colegios registran celulares como `+57 300 1 345 549` mientras que otros usan el formato `+57 3001345549`. Similarmente, los teléfonos fijos se encuentran escritos de maneras distintas, como `454 12 23` o `4541223`. Esta falta de uniformidad puede complicar la identificación de números telefónicos nuevos en el futuro.

En la versión actual, este problema no se ha solucionado. Por ello, los números de teléfono se guardan tal como se reciben en el campo `info_red_telefono_original`, para poder consultarlos más adelante.

In [30]:
red_academica['info_red_telefono_original'] = red_academica['teléfono']

In [31]:
red_academica['dane'] = red_academica['dane'].apply(clean_and_split_phones)

In [32]:
red_academica.head()

,jornada,dirección,correo,teléfono,horario de atención,nombre,idx,dane,red_telefono,info_red_telefono_original
0,"Mañana, Tarde, Nocturna",Carrera 18A # 187-75,insdiaquileoparrav1@educacionbogota.edu.co,6745973 - 6745973,8:00 a.m a 4:00 p.m.,Colegio Aquileo Parra,0,NaN,"[6745973, 6745973]",6745973 - 6745973
1,Jornada única,km 13 de la vía Bogotá - Choachí,escdielverjonbajoe2@redp.edu.co,NaN,NaN,IED Colegio El Verjon,0,NaN,NaN,NaN
2,"Mañana, Tarde",Calle 42 A Sur # 12 A - 27 Este,escdialtamirasuror4@educacionbogota.edu.co,2068596 / 3043999116,M y V 9:00 a.m. a 11:00 a.m. - 2:00 p.m. a 4:0...,Colegio Altamira Sur Oriental (IED),0,NaN,"[2068596, 3043999116]",2068596 / 3043999116
3,"Mañana, Tarde",Calle 24A Sur No 1A- 95 Este,cedmontebello4@educacionbogota.edu.co,2067680 - 3002067382,8:00 a.m. - 4:00 p.m.,Colegio Montebello (IED),0,NaN,"[2067680, 3002067382]",2067680 - 3002067382
4,"Mañana, Tarde",Diagonal 34 Sur # 2A-05 Este Barrio: Atenas.,NaN,2069970,Lunes a Viernes de 8:00 am - 4:30pm.,Colegio Atenas (IED),0,NaN,[2069970],2069970


Se ha observado que algunas instituciones educativas informan de múltiples códigos DANE. Para mejorar la eficiencia en la búsqueda por código, se requiere un ajuste en el formato de los datos. La celda de código a continuación se especializa en la detección y agrupación de estos códigos DANE, organizándolos en una lista para un acceso más sencillo y estructurado.

In [33]:
def unique_numeric_elements(lst):
    unique_nums = []
    if isinstance(lst, list):
        for item in lst:
            try:
                # Attempt to convert to integer
                num = int(item)
                # If successful and not already in list, add to unique_nums
                if num not in unique_nums:
                    unique_nums.append(num)
            except ValueError:
                # If conversion fails, skip the item
                continue
    return unique_nums

red_academica['dane'] = red_academica['dane'].apply(unique_numeric_elements)


In [34]:
red_academica['dane'].value_counts()

[]                                                          179
[111001015601]                                                2
[211001076346]                                                1
[111001083011]                                                1
[111001018058, 111001018031, 111001028355, 111001028304]      1
                                                           ... 
[111001010731]                                                1
[111001104388]                                                1
[111001094889]                                                1
[11127900061]                                                 1
[111001102199]                                                1
Name: dane, Length: 113, dtype: int64

In [35]:
red_academica = red_academica.explode('dane')

# Merge por DIAN

Esta sección está dedicada a la primera fusión de datos. Se procederá a buscar coincidencias entre los colegios de la base de datos principal y aquellos listados en Red Académica y Datos Colombia. Para ello, se utilizará el Código DANE como clave de emparejamiento


## 1. Datos Colombia -- Colegios de Bogotá


In [36]:
base_principal.head()

,nombre,direccion,telefono,base,id_dane,id_sirbe
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,KR 51 # 16 - 64 SUR,"[7209631, 2028884]",DANE,1.110010e+11,NaN
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,KR 70 B # 24 B - 35 SUR,"[4140218, 4140223]",DANE,1.110010e+11,NaN
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,KR 18 A # 187 - 67/65,"[6714615, 3043997208]",DANE,1.110010e+11,NaN
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,CL 15 A SUR # 6 - 25 ESTE,[6012464434],DANE,1.110010e+11,NaN
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,CL 90 A # 95 D - 56 -- KR 95 G # 90 A - 40,"[4345708, 4339567, 4339468]",DANE,1.110010e+11,NaN


In [37]:
colegios_bogota.head()

,id_institución,nombre_establecimiento,nombre_sede,estado_sede,código_establecimiento,código_sede,zona,nuevo_telefono,dirección,ciudad,departamento,modelos,grados,url,name,page_num
0,106399.0,Col Cambridge,Col Cambridge,Cierre Temporal,3.110010e+11,3.110010e+11,Urbana,[2158649],Tv 14 A 115 06,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/10...,Col Cambridge,1
1,113157.0,Jard Inf Margy,Jard Inf Margy,Cierre Definitivo,3.110011e+11,3.110011e+11,Urbana,"[3680192, 3690961]",Cr. 45 No.22c-51,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Jard Inf Margy,5
2,113374.0,Inst Scalas,Inst Scalas,Cierre Temporal,3.110011e+11,3.110011e+11,Urbana,[2398028],Cr. 6 No. 25-60 Sur,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Inst Scalas,5
3,116203.0,Colegio Tibabuyes Universal (Ied),Sede C,Nuevo-Activo,1.110010e+11,1.110011e+11,Urbana,NaN,Kr 107 139 78,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Sede C,5
4,116593.0,Col Gimn Carolin,Col Gimn Carolin,Cierre Temporal,3.110011e+11,3.110011e+11,Urbana,[7689768],Kr 6 C 90 50 Sur,"Bogotá, D.C.","Bogotá, D.C.",NaN,NaN,https://datoscolombia.com/escuelas-colegios/11...,Col Gimn Carolin,5


Se observa que del total de instituciones que se encuentran en la base principal, el 68% se encuentra en la base de datos a comparar. Esto es una buena señal para el emparejamiento.

In [38]:
base_principal.id_dane.isin(colegios_bogota_tel_and_id.código_establecimiento).sum()/len(base_principal)

0.6841411144081767

In [39]:
base_principal = base_principal.merge(colegios_bogota_tel_and_id[~colegios_bogota_tel_and_id['código_establecimiento'].isna()], how = 'left', left_on = 'id_dane', right_on = 'código_establecimiento')

Luego del merge, se tiene la base de datos principal con una columna nueva que indica los teléfonos que aparecen en la base de datos `colegios_bogota_tel_and_id`. Estos colegios pueden coincidir con algunos existentes en la `base_principal`.

Posteriormente se filtrarán únicamente los teléfonos nuevos para cada institución.

In [40]:
base_principal[['nombre', 'id_dane', 'telefono', 'nuevo_telefono']].head()

,nombre,id_dane,telefono,nuevo_telefono
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,1.110010e+11,"[7209631, 2028884]","[5650250, 7209631, 2022340, 2387091]"
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,1.110010e+11,"[4140218, 4140223]","[4140218, 4140223, 2048314, 2048323]"
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,1.110010e+11,"[6714615, 3043997208]","[6704916, 6745973, 6749258]"
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,1.110010e+11,[6012464434],NaN
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,1.110010e+11,"[4345708, 4339567, 4339468]","[4345708, 4339567, 2295726]"


## 2. Red Académica

Red Académica contiene una amplia gama de información sobre los colegios, incluyendo detalles como el correo electrónico, horarios de atención y jornadas escolares, que pueden ser valiosos para nuestro proyecto. Por el momento, nos centraremos exclusivamente en los datos de contacto, como teléfonos y correos electrónicos. No obstante, el código se puede adaptar posteriormente para incorporar cualquier información adicional que se requiera.


In [41]:
red_academica.head()

,jornada,dirección,correo,teléfono,horario de atención,nombre,idx,dane,red_telefono,info_red_telefono_original
0,"Mañana, Tarde, Nocturna",Carrera 18A # 187-75,insdiaquileoparrav1@educacionbogota.edu.co,6745973 - 6745973,8:00 a.m a 4:00 p.m.,Colegio Aquileo Parra,0,NaN,"[6745973, 6745973]",6745973 - 6745973
1,Jornada única,km 13 de la vía Bogotá - Choachí,escdielverjonbajoe2@redp.edu.co,NaN,NaN,IED Colegio El Verjon,0,NaN,NaN,NaN
2,"Mañana, Tarde",Calle 42 A Sur # 12 A - 27 Este,escdialtamirasuror4@educacionbogota.edu.co,2068596 / 3043999116,M y V 9:00 a.m. a 11:00 a.m. - 2:00 p.m. a 4:0...,Colegio Altamira Sur Oriental (IED),0,NaN,"[2068596, 3043999116]",2068596 / 3043999116
3,"Mañana, Tarde",Calle 24A Sur No 1A- 95 Este,cedmontebello4@educacionbogota.edu.co,2067680 - 3002067382,8:00 a.m. - 4:00 p.m.,Colegio Montebello (IED),0,NaN,"[2067680, 3002067382]",2067680 - 3002067382
4,"Mañana, Tarde",Diagonal 34 Sur # 2A-05 Este Barrio: Atenas.,NaN,2069970,Lunes a Viernes de 8:00 am - 4:30pm.,Colegio Atenas (IED),0,NaN,[2069970],2069970


In [42]:
red_academica_info = red_academica[['dane', 'red_telefono', 'correo', 'info_red_telefono_original']]

In [43]:
base_principal = base_principal.merge(red_academica_info[~red_academica_info.dane.isna()], how = 'left', left_on = 'id_dane', right_on = 'dane')

A continuación, se muestra un ejemplo de números telefónicos que aparecen con errores. Aunque el filtro aplicado originalmente no tenía como objetivo identificar estos casos, coincidentemente reveló los problemas mencionados previamente.

In [44]:
base_principal[(~base_principal.red_telefono.isna()) & (base_principal.nuevo_telefono.isna())] # Identificamos algunos que toca cambiar

,nombre,direccion,telefono,base,id_dane,id_sirbe,código_establecimiento,nuevo_telefono,dane,red_telefono,correo,info_red_telefono_original
584,COLEGIO LA FELICIDAD (IED)-LA FELICIDAD,CL 19 # 79 - 37,"[6478800, 3058141739]",DANE,111001800091.0,NaN,NaN,NaN,111001800091,"[57, 305, 8141739]",colfelicidad9@educacionbogota.edu.co,+57 305 8141739
593,COLEGIO LAS MARGARITAS (IED)-LAS MARGARITAS,KR 88 C # 43 - 53 SUR,[3016733388],DANE,111001800431.0,NaN,NaN,NaN,111001800431,[3016733388],collasmargaritas8@educacionbogota.edu.co,3016733388
596,COLEGIO EMMA REYES (IED)-EMMA REYES,TV 78 C # 6 D - 49,[2736112],DANE,111001800554.0,NaN,NaN,NaN,111001800554,"[310, 2967796]",col.emma.reyes@educacionbogota.edu.co,310 2967796


Se llevó a cabo una revisión de aquellos colegios que carecían de código DANE y para los cuales no se logró obtener información a través de Red Académica. Descubrimos que la mayoría de estos pertenecen a SIRBE (abarcando la totalidad de esta base de datos).

In [45]:
base_principal[(base_principal.id_dane.isna()) & (base_principal.info_red_telefono_original.isna())].base.value_counts()

SIRBE    469
DANE       1
Name: base, dtype: int64

In [46]:
base_principal[(base_principal.id_dane.isna()) & (base_principal.info_red_telefono_original.isna())]

,nombre,direccion,telefono,base,id_dane,id_sirbe,código_establecimiento,nuevo_telefono,dane,red_telefono,correo,info_red_telefono_original
2541,NaN,NaN,NaN,DANE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2546,8-JINF-SDIS-BARRANCAS-,KR 7 155 80,[6728319],SIRBE,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
2547,9-JINF-SDIS-VERBENAL-,KR 18 A 187 75,[6780889],SIRBE,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
2548,11-JINF-SDIS-BUENAVISTA-,CL 190 4 C 63,[2643517],SIRBE,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
2549,17-JINF-SDIS-JUAN XXIII-,CL 65 B 1 19,[3103689],SIRBE,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3029,MIS PRIMEROS TRAZOS,KR 87 D 39 10 SUR,[4500580],SIRBE,NaN,121788.0,NaN,NaN,NaN,NaN,NaN,NaN
3030,LA FELICIDAD,CL 49 A SUR 5 B 25,[4555585],SIRBE,NaN,121869.0,NaN,NaN,NaN,NaN,NaN,NaN
3031,MIRADOR MAGICO,DG 48 K BIS SUR 5 X 67,[2053637],SIRBE,NaN,122064.0,NaN,NaN,NaN,NaN,NaN,NaN
3032,HIJOS DE PALESTINA,KR 81 D 68 33 SUR,[7190705],SIRBE,NaN,122089.0,NaN,NaN,NaN,NaN,NaN,NaN


Un ejemplo de un colegio que no se encontró es el `COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN INFANTIL NACIONAL POPULAR No 2`, el cual ha cambiado de nombre varias veces en su historia. De hcho, la página de [Red Académica](https://www.redacademica.edu.co/colegios/colegio-tecnico-san-cristobal-sur-ied) de este colegio, indica que la sede que buscamos ya no existe como tal.

Con lo anterior, se motiva el emparejamiento por nombre.

# Merge by Nombre

Lo que buscamos es hacer emparejamientos de los colegios a partir del nombre. Lo haremos únicamente para los colegios que no tuvieron match por código DANE en ninguna de las bases anteriores.

In [47]:
schools_to_find = base_principal[(base_principal.nuevo_telefono.isna()) & (base_principal.info_red_telefono_original.isna())].copy()
schools_to_find = schools_to_find[['nombre', 'base']]

Podemos ver que en esta nueva aproximación, solo hacen falta 955 colegios por obtener un teléfono de alguna de las dos nuevas fuentes.

En esta base se encuentra la totalidad de colegios del SIRBE y 467 colegios de la base del DANE (18% de la base de DANE).

In [48]:
len(schools_to_find)

955

In [49]:
schools_to_find.base.value_counts()

SIRBE    469
DANE     467
Name: base, dtype: int64

In [50]:
schools_to_find.head()

,nombre,base
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,DANE
15,COLEGIO FABIO LOZANO SIMONELLI (IED)-DANUBIO AZUL,DANE
21,COLEGIO JORGE ELIECER GAITAN (IED)-JARDIN INFA...,DANE
45,COLEGIO FRANCISCO PRIMERO S.S. (IED)-SEDE B,DANE
46,COLEGIO TECNICO JOSE FELIX RESTREPO (IED)-JUAN...,DANE


Para el emparejamiento de datos, emplearemos una técnica sencilla de String Matching basada en la distancia de Levenshtein. Esta metodología se enfoca en determinar la cantidad mínima de ediciones necesarias para que una cadena de texto se asemeje a otra específica.

La cadena que requiera el menor número de modificaciones se considerará la más parecida. Por esta razón, es crucial normalizar los nombres de los colegios de manera homogénea. Esto implica trabajar con los nombres en minúsculas, eliminar duplicaciones innecesarias en los nombres (por ejemplo, nombres repetidos) y descartar el nombre de la sede específica de cada colegio.

In [51]:
schools_to_find['processed_name'] = schools_to_find['nombre'].str.split('-').str[0].str.lower()
schools_to_find = schools_to_find[~schools_to_find.processed_name.isna()]

In [52]:
# Los colegios de la base de SIRBE tienen un formato super extraño. Allí no podemos hacer esto que tenemos
schools_to_find.loc[ schools_to_find.base == 'SIRBE','processed_name'] = schools_to_find['nombre'].str.replace('-', ' ')

In [53]:
# Procesamos Texto en donde se busca
red_academica['nombre_processed'] = red_academica['nombre'].str.lower()
colegios_bogota['nombre_establecimiento_processed'] = colegios_bogota['nombre_establecimiento'].str.lower()

In [54]:
# Anteriormente observamos que había múltiples sedes por código DANE.
# Esto también sucede por nombre principal. Es necesario agrupar.
grouped = colegios_bogota.groupby('nombre_establecimiento_processed')['nuevo_telefono'].agg(merge_telefonos)
colegios_bogota_string_match = grouped.to_frame()
colegios_bogota_string_match.reset_index(inplace=True)

## Emparejamiento

Desarrollamos un sistema de puntuación combinado para evaluar la similitud. Primero, calculamos un puntaje de similitud estándar basándonos únicamente en el número de ediciones necesarias. Luego, generamos un segundo puntaje que implica tokenizar ambas cadenas de entrada, ordenar los tokens alfabéticamente y calcular la similitud usando el Fuzz Ratio entre las listas de tokens ordenadas.

Al promediar estos dos puntajes, obtenemos el puntaje final de coincidencia, que refleja la similitud general entre las cadenas comparadas.

In [55]:
def combined_scorer(s1, s2):
  """
  Crea un score combinado para similitud entre strings.
  """
  token_sort = fuzz.token_sort_ratio(s1, s2)
  normal_ratio = fuzz.ratio(s1, s2)
  return (token_sort + normal_ratio)*0.5

Las listas de nombres de colegios a comparar son los nombres procesados de cada una de las bases de datos:

In [56]:
# Colegios Red Académica
comparisons_list_red = red_academica.nombre_processed.values.tolist()

# Colegios Datos Colombia
comparisons_list_bogota = colegios_bogota_string_match.nombre_establecimiento_processed.values.tolist()


A continuación, presentamos la definición de una función diseñada para identificar los colegios con mayor grado de similitud. Esta función está configurada para seleccionar aquellos colegios cuya coincidencia alcance o supere el 95% de similitud.

La función `obtain_matching_by_list` recibe una lista para comparar y, mediante la función `find_matches`, compara el nombre procesado de cada colegio con la lista dada. Utiliza el `combined_scorer` definido anteriormente para calcular la similitud. Si la similitud es igual o superior al 95%, devuelve el nombre del colegio correspondiente.

In [57]:

def obtain_matching_by_list(comparison_list):
  def find_matches(row):
    school_to_find = row.processed_name
    result = process.extractOne(school_to_find, comparison_list, scorer = combined_scorer)
    if result[1] >= 95:
      return result[0]
  return find_matches

In [58]:
schools_to_find['red_match'] = schools_to_find.apply(obtain_matching_by_list(comparisons_list_red), axis = 1)
schools_to_find['colegios_bta_match'] = schools_to_find.apply(obtain_matching_by_list(comparisons_list_bogota), axis = 1)

Tras completar el proceso de emparejamiento, obtuvimos los siguientes resultados:

- Se emparejaron exitosamente 312 colegios de la base de Colegios Bogotá.
- Se logró emparejar 234 colegios de la base de Red Académica.
- En total, ahora disponemos de información combinada para 341 colegios, emparejados con al menos una de estas dos bases de datos.


Esta metodología tiene potencial para ser ampliada en el futuro, incorporando más bases de datos relacionadas con información de colegios.

In [59]:
(1 - schools_to_find.colegios_bta_match.isna()).sum()

312

In [60]:
(1 - schools_to_find.red_match.isna()).sum()

234

En principio recuperaríamos información para 341 colegios que antes no tendrían, siendo en su totalidad del DANE

In [61]:
len(schools_to_find[~((schools_to_find.red_match.isna()) & (schools_to_find.colegios_bta_match.isna()))])

341

Sin embargo, la totalidad de los emparejamientos encontrados fue únicamente para los colegios de la base del DANE.

In [62]:
schools_to_find[~((schools_to_find.red_match.isna()) & (schools_to_find.colegios_bta_match.isna()))].base.value_counts()

DANE    341
Name: base, dtype: int64

## Recuperación de Información

Después de realizar el emparejamiento con las bases de datos mencionadas, el siguiente paso es recuperar la información de contacto de los colegios emparejados. Este proceso, centrado en fusionar las bases originales utilizando las parejas identificadas y generar un archivo .csv delimitado por `;` para su uso, no se detallará aquí, ya que su propósito es meramente técnico y funcional.

In [63]:
schools_to_find.head()

,nombre,base,processed_name,red_match,colegios_bta_match
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,DANE,colegio tecnico san cristobal sur (ied),colegio técnico san cristobal sur (ied),None
15,COLEGIO FABIO LOZANO SIMONELLI (IED)-DANUBIO AZUL,DANE,colegio fabio lozano simonelli (ied),colegio fabio lozano simonelli ied,colegio fabio lozano simonelli (ied)
21,COLEGIO JORGE ELIECER GAITAN (IED)-JARDIN INFA...,DANE,colegio jorge eliecer gaitan (ied),None,colegio jorge eliecer gaitan (ied)
45,COLEGIO FRANCISCO PRIMERO S.S. (IED)-SEDE B,DANE,colegio francisco primero s.s. (ied),colegio francisco primero s.s. (ied),None
46,COLEGIO TECNICO JOSE FELIX RESTREPO (IED)-JUAN...,DANE,colegio tecnico jose felix restrepo (ied),None,None


In [64]:
# Seleccionamos los colegios que consiguieron emparejamiento
schools_found = schools_to_find[~((schools_to_find.red_match.isna()) & (schools_to_find.colegios_bta_match.isna()))].copy()

# Información a Mergear
colegios_bogota_by_name = colegios_bogota_string_match[['nombre_establecimiento_processed', 'nuevo_telefono']].copy()
colegios_red_by_name = red_academica[['nombre_processed', 'red_telefono', 'correo', 'info_red_telefono_original']].copy()

In [65]:
# Realizamos el merge con las bases de donde se encontró información
schools_found = schools_found.merge(colegios_bogota_by_name, how = 'left', left_on = 'colegios_bta_match', right_on = 'nombre_establecimiento_processed')
schools_found = schools_found.merge(colegios_red_by_name, how = 'left', left_on = 'red_match', right_on = 'nombre_processed')

In [66]:
schools_found = schools_found[['nombre','nuevo_telefono', 'red_telefono', 'correo', 'info_red_telefono_original']]

In [67]:
schools_found.head()

,nombre,nuevo_telefono,red_telefono,correo,info_red_telefono_original
0,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,NaN,[3002067977],cedsancristobasu4@educacionbogota.edu.co,3002067977
1,COLEGIO FABIO LOZANO SIMONELLI (IED)-DANUBIO AZUL,"[7692604, 7647342, 7621132, 7621084]","[7621132, 7621084, , , 7692604, , , 7647342, , ]",fabiolozanosimonelli@gmail.com,7621132 / 7621084 Sede A - 7692604 Sede B - 76...
2,COLEGIO JORGE ELIECER GAITAN (IED)-JARDIN INFA...,"[6310047, 6603278, 3119495, 3119496, 2502503]",NaN,NaN,NaN
3,COLEGIO FRANCISCO PRIMERO S.S. (IED)-SEDE B,NaN,"[3491208, 2126019]",colfranciscoprimero@educacionbogota.edu.co,3491208 / 2126019
4,COLEGIO MANUELA BELTRAN (IED)-ANTONIO NARIÑO,"[2559708, 2114894, 2683425]","[, 2552363, 2494003, , , 3407320]",iedmanuelabeltran13@educacionbogota.edu.co,Sede A2552363/ 2494003/ Sede B 3407320


In [68]:
base_principal = base_principal.merge(schools_found, on = 'nombre', how = 'left')

In [69]:
base_principal.head()

,nombre,direccion,telefono,base,id_dane,id_sirbe,código_establecimiento,nuevo_telefono_x,dane,red_telefono_x,correo_x,info_red_telefono_original_x,nuevo_telefono_y,red_telefono_y,correo_y,info_red_telefono_original_y
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,KR 51 # 16 - 64 SUR,"[7209631, 2028884]",DANE,111001000078.0,NaN,1.110010e+11,"[5650250, 7209631, 2022340, 2387091]",111001000078,"[2028772, , 101, 3002067416, , ]",innaldeculturapopu16@educacionbogota.edu.co,2028772 Ext 101-3002067416 Sede Administrativa,NaN,NaN,NaN,NaN
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,KR 70 B # 24 B - 35 SUR,"[4140218, 4140223]",DANE,111001000124.0,NaN,1.110010e+11,"[4140218, 4140223, 2048314, 2048323]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,KR 18 A # 187 - 67/65,"[6714615, 3043997208]",DANE,111001000132.0,NaN,1.110010e+11,"[6704916, 6745973, 6749258]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,CL 15 A SUR # 6 - 25 ESTE,[6012464434],DANE,111001000213.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[3002067977],cedsancristobasu4@educacionbogota.edu.co,3002067977
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,CL 90 A # 95 D - 56 -- KR 95 G # 90 A - 40,"[4345708, 4339567, 4339468]",DANE,111001000272.0,NaN,1.110010e+11,"[4345708, 4339567, 2295726]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
def coalesce_field(df, column):
  return df[column + '_x'].combine_first(df[column + '_y'])

In [71]:
columns_to_coalesce = [x.replace('_x', '') for x in base_principal.columns if x.find('_x') != -1]

for column in columns_to_coalesce:
    base_principal[column] = base_principal[column + '_x'].combine_first(base_principal[column + '_y'])
    base_principal.drop([column + '_x', column + '_y'], axis=1, inplace=True)


In [72]:
import numpy as np

def find_unique_elements(row):
    # Treat NaN as an empty list for 'telefono', 'nuevo_telefono', and 'red_telefono'
    telefono = [] if isinstance(row['telefono'], float) and np.isnan(row['telefono']) else row['telefono']
    nuevo_telefono = [] if isinstance(row['nuevo_telefono'], float) and np.isnan(row['nuevo_telefono']) else row['nuevo_telefono']
    red_telefono = [] if isinstance(row['red_telefono'], float) and np.isnan(row['red_telefono']) else row['red_telefono']

    # Combine 'nuevo_telefono' and 'red_telefono' into one list
    combined_telefonos = nuevo_telefono + red_telefono

    # Return the unique elements in combined list not in 'telefono'
    return [item for item in combined_telefonos if item not in telefono]

# Apply the function to each row
base_principal['new_available_phones'] = base_principal.apply(find_unique_elements, axis=1)

In [73]:
base_principal.head()

,nombre,direccion,telefono,base,id_dane,id_sirbe,código_establecimiento,dane,nuevo_telefono,red_telefono,correo,info_red_telefono_original,new_available_phones
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,KR 51 # 16 - 64 SUR,"[7209631, 2028884]",DANE,111001000078.0,NaN,1.110010e+11,111001000078,"[5650250, 7209631, 2022340, 2387091]","[2028772, , 101, 3002067416, , ]",innaldeculturapopu16@educacionbogota.edu.co,2028772 Ext 101-3002067416 Sede Administrativa,"[5650250, 2022340, 2387091, 2028772, , 101, 30..."
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,KR 70 B # 24 B - 35 SUR,"[4140218, 4140223]",DANE,111001000124.0,NaN,1.110010e+11,NaN,"[4140218, 4140223, 2048314, 2048323]",NaN,NaN,NaN,"[2048314, 2048323]"
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,KR 18 A # 187 - 67/65,"[6714615, 3043997208]",DANE,111001000132.0,NaN,1.110010e+11,NaN,"[6704916, 6745973, 6749258]",NaN,NaN,NaN,"[6704916, 6745973, 6749258]"
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,CL 15 A SUR # 6 - 25 ESTE,[6012464434],DANE,111001000213.0,NaN,NaN,NaN,NaN,[3002067977],cedsancristobasu4@educacionbogota.edu.co,3002067977,[3002067977]
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,CL 90 A # 95 D - 56 -- KR 95 G # 90 A - 40,"[4345708, 4339567, 4339468]",DANE,111001000272.0,NaN,1.110010e+11,NaN,"[4345708, 4339567, 2295726]",NaN,NaN,NaN,[2295726]


In [74]:
phone_columns = ['telefono', 'nuevo_telefono', 'red_telefono', 'new_available_phones']
base_principal[phone_columns] = base_principal[phone_columns].fillna('[]')
base_principal[phone_columns] = base_principal[phone_columns].astype(str)


In [75]:
base_principal = base_principal.drop_duplicates()

In [76]:
# Convertir columnas de teléfono nuevamente a listas
for column in phone_columns:
    if isinstance(base_principal[column].iloc[0], str) and base_principal[column].iloc[0].startswith('['):
        base_principal[column] = base_principal[column].apply(lambda x: ast.literal_eval(x) if x.startswith('[') else x)

<ipython-input-76-5672afa28805>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_principal[column] = base_principal[column].apply(lambda x: ast.literal_eval(x) if x.startswith('[') else x)


In [77]:
def replace_empty_with_nan(element):
    if element == []:
        return np.nan
    return element

for column in phone_columns:
  base_principal[column] = base_principal[column].apply(replace_empty_with_nan)

In [78]:
base_principal.head()

,nombre,direccion,telefono,base,id_dane,id_sirbe,código_establecimiento,dane,nuevo_telefono,red_telefono,correo,info_red_telefono_original,new_available_phones
0,COLEGIO DE CULTURA POPULAR (IED)-DE CULTURA PO...,KR 51 # 16 - 64 SUR,"[7209631, 2028884]",DANE,111001000078.0,NaN,1.110010e+11,111001000078,"[5650250, 7209631, 2022340, 2387091]","[2028772, , 101, 3002067416, , ]",innaldeculturapopu16@educacionbogota.edu.co,2028772 Ext 101-3002067416 Sede Administrativa,"[5650250, 2022340, 2387091, 2028772, , 101, 30..."
1,COLEGIO CARLOS ARANGO VELEZ (IED)-CARLOS ARANG...,KR 70 B # 24 B - 35 SUR,"[4140218, 4140223]",DANE,111001000124.0,NaN,1.110010e+11,NaN,"[4140218, 4140223, 2048314, 2048323]",NaN,NaN,NaN,"[2048314, 2048323]"
2,COLEGIO AQUILEO PARRA (IED)-AQUILEO PARRA,KR 18 A # 187 - 67/65,"[6714615, 3043997208]",DANE,111001000132.0,NaN,1.110010e+11,NaN,"[6704916, 6745973, 6749258]",NaN,NaN,NaN,"[6704916, 6745973, 6749258]"
3,COLEGIO TECNICO SAN CRISTOBAL SUR (IED)-JARDIN...,CL 15 A SUR # 6 - 25 ESTE,[6012464434],DANE,111001000213.0,NaN,NaN,NaN,NaN,[3002067977],cedsancristobasu4@educacionbogota.edu.co,3002067977,[3002067977]
4,COLEGIO INSTITUTO TECNICO LAUREANO GOMEZ (IED)...,CL 90 A # 95 D - 56 -- KR 95 G # 90 A - 40,"[4345708, 4339567, 4339468]",DANE,111001000272.0,NaN,1.110010e+11,NaN,"[4345708, 4339567, 2295726]",NaN,NaN,NaN,[2295726]


In [79]:
base_principal.to_csv('base_principal_complementada.csv', sep = ';', index = False)